In [16]:
import glob
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
from scipy import ndimage
import pickle
import nibabel as nib
import random

from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list


In [3]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
data_base_path = '/scratch/zq415/grammar_cor/mutant_detect/data'
data_folder_list = ['20180419_newdata_nii_with_filtered', 'new_data_20180522_nii', 'organized_data_nii']

In [4]:
save_name = 'All_data_112_64_64.pickle'
with open(os.path.join(os.getcwd(),'data',save_name), "rb") as input_file:
    all_train_data = pickle.load(input_file)

In [5]:
mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]

group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])
single_mutant = [i for i in range(len(mutant_names)) if i not in group_list]

test_mut_names = []
for i in range(1,len(mutant_group),6):
    for ii in range(len(mutant_group[i])):
        test_mut_names.append(mutant_names[mutant_group[i][ii]])

for i in range(1,len(single_mutant),6):
    test_mut_names.append(mutant_names[single_mutant[i]])
    
print(len(test_mut_names),test_mut_names)


18 ['20170207_En1_E13_E3_Mut', '20170207_En1_E13_E3c_Mut', '20170207_En1_E13_E3d_Mut', '20161206_En1_E13_E3a', '20161206_En1_E13_E3b', '20161206_En1_E13_E3c', '20161206_En1_E13_E3d', '20180131_En1_E12_E5a_Mut', '20180131_En1_E12_E5b', '20180131_En1_E12_E5c_Mut', '20180202_En1_E14_E3a_Mut', '20180202_En1_E14_E3b_Mut', '20171211_En1_E10_E1a', '20170619_En1_E12_E3a', '20170706_En1_E13_E11a_Mut_reg', '20171009_En1_E12_E10a', '20171211_En1_E13_E5a_Mut_Ext', '20180201_En1_M2_E13_E4a']


In [6]:
# for i in range(len(all_train_data)):
#     if all_train_data[i][2] in mutant_names:
#         img_nft = nib.Nifti1Image(all_train_data[i][3],np.eye(4))
#         img_save_data_path = './img/{}.nii'.format(all_train_data[i][2])
#         nib.save(img_nft,img_save_data_path)

In [7]:
train_data = []
test_data = []

for i in range(len(all_train_data)):
    if all_train_data[i][2] in mutant_names:
        if all_train_data[i][2] in test_mut_names:
            test_data.append((all_train_data[i][3]-0.5, 0 ))
        else:
            train_data.append((all_train_data[i][3]-0.5,0))
    else:
        random.seed(i*8)
        if random.uniform(0,1) < 0.16:
            test_data.append((all_train_data[i][3]-0.5,1))
        else:
            train_data.append((all_train_data[i][3]-0.5,1))

print(len(test_data))
print(len(train_data))


96
471


In [8]:
#img_nft = nib.Nifti1Image(np.squeeze(all_train_data[193]),np.eye(4))
#img_save_data_path = './img/mul_img.nii'
#nib.save(img_nft,img_save_data_path)

In [9]:
from torch.utils.data import Dataset, DataLoader

class Mouse_sub_volumes(Dataset):
    """Mouse sub-volumes BV dataset."""

    def __init__(self, all_data , transform=None):
        """
        Args:
            all_whole_volumes: Contain all the padded whole BV volumes as a dic
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.all_data = all_data
        self.transform = transform
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self, num):
        
        current_img, label = self.all_data[num]
        
        img = np.float32(current_img[np.newaxis,...])
        sample = {'image': img, 'label': label}
        return sample

In [10]:
class Flip(object):
    
    """
    Flip the image for data augmentation, but prefer original image.
    """
    
    def __init__(self,ori_probability=0.20):
        self.ori_probability = ori_probability

    def __call__(self, sample):
        if random.uniform(0,1) < self.ori_probability:
            return sample
        else:
            img, label = sample['image'], sample['label']
            random_choise1=random.choice([1,2,3,4,5,6,7,8])
            img[0,...] = {1: lambda x: x,
                          2: lambda x: x[::-1,:,:],
                          3: lambda x: x[:,::-1,:],
                          4: lambda x: x[:,:,::-1],
                          5: lambda x: x[::-1,::-1,:],
                          6: lambda x: x[::-1,:,::-1],
                          7: lambda x: x[:,::-1,::-1],
                          8: lambda x: x[::-1,::-1,::-1]
                          }[random_choise1](img[0,...])
        return {'image': img, 'label': label}

In [11]:
# import plotly.plotly as py
# import plotly.graph_objs as go
# import numpy as np
# import cufflinks as cf

# cf.go_offline()

# bv_map = all_train_data[0][3]
# x,y,z = np.where(bv_map==1)

# trace = go.Mesh3d(x=x,y=y,z=z,
#                    alphahull=5,
#                    opacity=0.4,
#                    color='#00FFFF')
# py.iplot([trace])

In [12]:
# import plotly.plotly as py
# import plotly.graph_objs as go
# import numpy as np

# bv_map = all_train_data[0][3]
# x,y,z = np.where(bv_map==1)
# #pts=np.loadtxt('mesh_dataset.txt')
# #x,y,z=zip(*pts)

# trace = go.Mesh3d(x=x,y=y,z=z,color='#FFB6C1',opacity=0.50)
# py.iplot([trace])

In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class VGG_net(nn.Module):
    def __init__(self,conv_drop_rate=0.15,linear_drop_rate=0.4):
        super(VGG_net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=8, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv1_bn = nn.BatchNorm3d(8)
        self.conv2 = nn.Conv3d(in_channels=8, out_channels=8, kernel_size=3,stride=1,padding=2, dilation=2)
        self.conv2_bn = nn.BatchNorm3d(8)
        self.pool1 = nn.MaxPool3d(2, 2)
        self.dropout1 = nn.Dropout3d(conv_drop_rate)
        
        self.conv3 = nn.Conv3d(in_channels=8, out_channels=12, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv3_bn = nn.BatchNorm3d(12)
        self.conv4 = nn.Conv3d(in_channels=12, out_channels=12, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv4_bn = nn.BatchNorm3d(12)
        self.pool2 = nn.MaxPool3d(2, 2)
        self.dropout2 = nn.Dropout3d(conv_drop_rate)
        
        self.conv5 = nn.Conv3d(in_channels=12, out_channels=18, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv5_bn = nn.BatchNorm3d(18)
        self.conv6 = nn.Conv3d(in_channels=18, out_channels=18, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv6_bn = nn.BatchNorm3d(18)
        self.pool3 = nn.MaxPool3d(2, 2)
        self.dropout3 = nn.Dropout3d(conv_drop_rate)
        
        self.conv7 = nn.Conv3d(in_channels=18, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv7_bn = nn.BatchNorm3d(24)
        self.conv8 = nn.Conv3d(in_channels=24, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv8_bn = nn.BatchNorm3d(24)
        self.pool4 = nn.MaxPool3d(2, 2)
        self.dropout4 = nn.Dropout3d(conv_drop_rate)
        
        self.fc1 = nn.Linear(7*4*4*24, 128)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.dropout5 = nn.Dropout(linear_drop_rate)
        self.fc2 = nn.Linear(128, 2)
        
        
        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=5, stride=1, padding=2,dilation=1),
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2),
            
            nn.Conv3d(8, 8, kernel_size=5, stride=1, padding=2,dilation=1),
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2),
            
            
            nn.Conv3d(8, 8, kernel_size=5, stride=1, padding=2,dilation=1),
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2),
            
            nn.Conv3d(8, 8, kernel_size=5, stride=1, padding=2,dilation=1),
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2)
        )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(8 * 7 * 4 * 4, 48),
            nn.ReLU(True),
            nn.Linear(48, 4 * 3)
        )

        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], dtype=torch.float))


        
    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 8 * 7 * 4 *4)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 3, 4)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x
    
    
    def forward(self, x):
        # transform the input
        x = self.stn(x)
        
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = self.dropout1(self.pool1(self.conv2_bn(F.relu(self.conv2(x)))))
        
        x = self.conv3_bn(F.relu(self.conv3(x)))
        x = self.dropout2(self.pool2(self.conv4_bn(F.relu(self.conv4(x)))))
        
        x = self.conv5_bn(F.relu(self.conv5(x)))
        x = self.dropout3(self.pool3(self.conv6_bn(F.relu(self.conv6(x)))))
        
        x = self.conv7_bn(F.relu(self.conv7(x)))
        x = self.dropout4(self.pool4(self.conv8_bn(F.relu(self.conv8(x)))))
        
        x = x.view(-1, 7*4*4*24)
        x = self.dropout5(self.fc1_bn(F.relu(self.fc1(x))))
        x = self.fc2(x)
        return x


In [22]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        i_batch += 1
        if i_batch % 10 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/10))
            running_loss = 0.0

def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy()))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

def get_confusion_matrix(true_predicted_labels):
    cross_table = np.zeros([2,2])
    mut_to_nor = []
    nor_to_mul = []
    test_dic = true_predicted_labels
    for i in range(len(test_dic)):
        if test_dic[i][0] ==0 and test_dic[i][1] ==0:
            cross_table[0,0] += 1
        elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
            cross_table[0,1] += 1
            mut_to_nor.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
            cross_table[1,0] += 1
            nor_to_mul.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
            cross_table[1,1] += 1
    print(cross_table)
    print(mut_to_nor)
    print(nor_to_mul)

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

num_epochs = 350
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([3.5,1.0]).to(device))

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
print('choose SGD as optimizer')
#optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
#print('choose Adam as optimizer')

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for epoch in range(num_epochs):
    scheduler.step()
    
    Mouse_dataset = Mouse_sub_volumes(train_data, transform=transforms.Compose([Flip()]))
    dataloader = DataLoader(Mouse_dataset, batch_size=12, shuffle=True, num_workers=4, drop_last = True)
    train(net, device, dataloader, optimizer, criterion, epoch)
    if (epoch + 1) % 50 == 0:
        print('epoch {} train accuracy: '.format(epoch+1))
        train_Mouse_dataset = Mouse_sub_volumes(train_data)
        train_dataloader = DataLoader(train_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        train_dic = test(net, device, train_dataloader)
        get_confusion_matrix(train_dic)
        
        print("-------------------")
        print('epoch {} test accuracy: '.format(epoch+1))
        test_Mouse_dataset = Mouse_sub_volumes(test_data)
        test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        test_dic = test(net, device, test_dataloader)
        get_confusion_matrix(test_dic)
        
        torch.save(net.state_dict(), './model/mut_clas_2019_01_17_e{}.pth'.format(epoch+1))

Let's use 2 GPUs!
There are 463974 parameters in the model
choose SGD as optimizer


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x2b5a826490b8>>
Traceback (most recent call last):
  File "/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiproce

ValueError: too many values to unpack (expected 4)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

Let's use 2 GPUs!
There are 395298 parameters in the model


In [15]:
def test_with_probability(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            outputs = F.softmax(outputs)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy(), outputs.cpu().numpy()[0,0], outputs.cpu().numpy()[0,1]))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

In [16]:
#torch.save(net.state_dict(), './model/mut_clas_2019_01_15.pth')

In [17]:
net.load_state_dict(torch.load('./model/mut_clas_2019_01_17_e350.pth'))

In [18]:
# print('train accuracy: ')
# Mouse_dataset = Mouse_sub_volumes(all_train_data,train_idx)
# train_dataloader = DataLoader(Mouse_dataset, batch_size=128,
#                         shuffle=False, num_workers=4)
# test(net, device, train_dataloader)

In [19]:
print('test accuracy: ')
Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

test_dic = test_with_probability(net, device, test_dataloader)

test accuracy: 


/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


total_num:96, test accuracy:0.9375, positive_acc:0.9487179487179487, negative_acc:0.8888888888888888


In [20]:
cross_table = np.zeros([2,2])
mut_to_nor = []
nor_to_mul = []

for i in range(len(test_dic)):
    if test_dic[i][0] ==0 and test_dic[i][1] ==0:
        cross_table[0,0] += 1
    elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
        cross_table[0,1] += 1
        mut_to_nor.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
        cross_table[1,0] += 1
        nor_to_mul.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
        cross_table[1,1] += 1
print(cross_table)
print(mut_to_nor)
print(nor_to_mul)

[[ 16.   2.]
 [  4.  74.]]
[0, 90]
[16, 32, 45, 46]


In [21]:
test_dic

[(array([0]), array([1]), 0.014204379, 0.98579562),
 (array([1]), array([1]), 0.008861592, 0.9911384),
 (array([1]), array([1]), 0.0020978176, 0.99790215),
 (array([1]), array([1]), 0.00019172154, 0.99980825),
 (array([1]), array([1]), 0.0045706001, 0.9954294),
 (array([1]), array([1]), 0.0028623149, 0.99713767),
 (array([1]), array([1]), 9.5042788e-06, 0.99999046),
 (array([1]), array([1]), 0.0032477456, 0.99675226),
 (array([1]), array([1]), 0.0064804694, 0.99351948),
 (array([0]), array([0]), 0.59785759, 0.40214238),
 (array([1]), array([1]), 0.00016109479, 0.99983883),
 (array([1]), array([1]), 0.0001081415, 0.99989188),
 (array([1]), array([1]), 0.0017415026, 0.99825853),
 (array([1]), array([1]), 0.00043563324, 0.99956435),
 (array([1]), array([1]), 0.01604772, 0.98395222),
 (array([1]), array([1]), 0.004320879, 0.9956792),
 (array([1]), array([0]), 0.89634043, 0.10365963),
 (array([1]), array([1]), 0.0070954543, 0.9929046),
 (array([1]), array([1]), 0.0014228453, 0.99857712),
 (

In [22]:
for i in mut_to_nor:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
    img_save_data_path = './img/mul_img{}.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

0
90


In [23]:
for i in nor_to_mul:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
    img_save_data_path = './img/nor_img{}.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

16
32
45
46


In [44]:
for i in range(len(test_data)):
    if test_data[i][1] == 0:
        img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
        img_save_data_path = './img/mul_img{}.nii'.format(i)
        nib.save(img_nft,img_save_data_path)

In [37]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Make input tensor require gradient
    X.requires_grad_()
    ##############################################################################
    # Perform a forward and backward pass through the model to compute the gradient 
    # of the correct class score with respect to each input image. You first want 
    # to compute the loss over the correct scores (we'll combine losses across a batch
    # by summing), and then compute the gradients with a backward pass.
    ##############################################################################
    scores = model(X)
    
    # Get the correct class computed scores.
    scores = scores.gather(1, y.view(-1, 1)).squeeze()  
    
    # Backward pass, need to supply initial gradients of same tensor shape as scores.
    scores.backward(torch.tensor(10.0).cuda(device))
    
    # Get gradient for image.
    saliency = X.grad.data
    
    # Convert from 3d to 1d.
    saliency = saliency.abs()
    saliency = saliency.squeeze()
    ##############################################################################
    return saliency

In [42]:
test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']  
    inputs = inputs.to(device)
    labels = labels.to(device)
    saliency = compute_saliency_maps(inputs, labels, net)
    
    max_value = torch.max(saliency)
    saliency[saliency >= (max_value*0.2)] = 1
    saliency[saliency < (max_value*0.2)] = 0
    
    img_nft = nib.Nifti1Image(np.squeeze(inputs.cpu().detach().numpy()+0.5),np.eye(4))
    img_save_data_path = './saliency_map/img_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(img_nft,img_save_data_path)
    
    saliency_nft = nib.Nifti1Image(np.squeeze(saliency.cpu().numpy()),np.eye(4))
    saliency_save_data_path = './saliency_map/salency_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(saliency_nft,saliency_save_data_path)
        

In [40]:
torch.max(saliency).cpu()

tensor(0.2944)

In [41]:
max_value = torch.max(saliency)
saliency[saliency >= (max_value*0.2)] = 1
saliency[saliency < (max_value*0.2)] = 0